In [ ]:
import math
import statistics

import pandas as pd
import networkx as nx

In [ ]:
# Load the records table with cleaned up country names as the primary data frame
records_path = "../data/records-renamed.csv"
df = pd.read_csv(records_path, index_col='ID', keep_default_na=False)

## Helpers

In [ ]:
# Helper constants
GA = 'General Assembly'
SC = 'Security Council'
START = 1946
END = 2024

In [ ]:
# Filter DFs with different parameters
def date_is_between(date, start_year, end_year):
    return date.str[0:4].astype(int).between(start_year, end_year)

def filter(df, body, start_year, end_year):
    return df[(df['Body'] == body) & (date_is_between(df['Date'], start_year, end_year))]

## Connect Countries

In [ ]:
vote_weights = {
    'Y': {
        'Y': 1.0,
        'N': 0.0,
        'A': 0.5,
        'X': 0.5
    },
    'N': {
        'Y': 0.0,
        'N': 1.0,
        'A': 0.5,
        'X': 0.5
    },
    'A': {
        'Y': 0.5,
        'N': 0.5,
        'A': 1.0,
        'X': 1.0
    },
    'X': {
        'Y': 0.5,
        'N': 0.5,
        'A': 1.0,
        'X': 1.0
    }
}

In [ ]:
def connect_countries(net, country1, country2, vote1, vote2):
    vote_weight = vote_weights[vote1][vote2]
    if net.has_edge(country1, country2):
        total = net[country1][country2]['total']
        points = net[country1][country2]['points']
        
        net[country1][country2]['total'] = total + 1
        net[country1][country2]['points'] = points + vote_weight
    else:
        net.add_edge(country1, country2, points = vote_weight, total=1)
        
def connect_voting_points(net, voting_point_1, voting_point_2):
    vote_1 = voting_point_1[0]
    vote_2 = voting_point_2[0]

    country_1 = voting_point_1[2:]
    country_2 = voting_point_2[2:]

    connect_countries(net, country_1, country_2, vote_1, vote_2)

In [ ]:
# Connect the countries and calculate their total agreement
def print_progress(finished, total, next_to_print):
    progress = math.floor(finished * 100 / total)
    if progress >= next_to_print:
        print('Progress: {}%'.format(progress))
        return next_to_print + 5
    else:
        return next_to_print

def calculate_agreement(net):
    for edge in net.edges(data=True):
        edge[2]['agreement'] = edge[2]['points'] / edge[2]['total']

def calculate_weight(net):
    for edge in net.edges(data=True):
        edge[2]['weight'] = edge[2]['agreement']

def connect_all_countries(net, body, start_year, end_year):
    df_wip = df[(df['Voting Data'] != 'Concensus') & (df['Voting Data'] != 'N/A')]
    df_wip = filter(df_wip, body, start_year, end_year)

    finished = 0
    next_to_print = 0
    total = df_wip.shape[0]

    for index, row in df_wip.iterrows():
        voting_points = row['Voting Data'].split(';')
        for i in range(0, len(voting_points)):
            for j in range(i + 1, len(voting_points)):
                connect_voting_points(net, voting_points[i], voting_points[j])

        finished += 1
        next_to_print = print_progress(finished, total, next_to_print)

    calculate_agreement(net)
    calculate_weight(net)

## Create Graphs Helpers

In [ ]:
country_coord_path = "./country-coord.csv"
country_coords = pd.read_csv(country_coord_path, index_col='Country')[['Longitude (average)', 'Latitude (average)']]
country_coords.index= country_coords.index.str.upper()

In [ ]:
country_size_path = "./country-size.csv"
country_sizes = pd.read_csv(country_size_path, index_col='Country')[['Area']]
country_sizes.index = country_sizes.index.str.upper()
country_sizes.index = country_sizes.index.str.strip()

In [ ]:
def add_coordinates(net):
    for country in net:
        if country in country_coords.index:
            country_coord = country_coords.loc[country]
            net.nodes[country]['x'] = country_coord['Longitude (average)'] * 10
            net.nodes[country]['y'] = country_coord['Latitude (average)'] * 10
        else:
            print('No coords found for ' + country)

In [ ]:
def add_sizes(net):
    for country in net:
        if country in country_sizes.index:
            size = country_sizes.loc[country]['Area'].item()
            net.nodes[country]['size'] = size
        else:
            print('No size found for ' + country)

In [ ]:
def save_graph(net, name):
    output_path = "../graphs/{}.gml".format(name)
    nx.write_gml(net, output_path)

In [ ]:
def create_graph(name, body, start_year, end_year):
    net = nx.Graph()
    
    connect_all_countries(net, body, start_year, end_year)
    add_coordinates(net)
    add_sizes(net)
    
    save_graph(net, name)
    
    return net

In [ ]:
def create_filled_graph(name, source_net):
    net = source_net.copy()
    
    all_weights = map(lambda edge: edge[2]['weight'], net.edges(data=True))
    median_weight = statistics.median(all_weights)
    print(f'Using weight as fill: {median_weight}')
    
    for countries in nx.non_edges(net_ga_all):
        net.add_edge(countries[0], countries[1], points=0.0, total=0, agreement=0.0, weight=median_weight)
    
    save_graph(net, name)

In [ ]:
def create_graphs_from_periods(periods, infix = ""):
    for period in periods:
        start = period[0]
        end = period[1]

        name = f'net_ga_{infix}_{start}_{end}' if len(infix) > 0 else f'net_ga_{start}_{end}'
        filled_name = f'{name}_filled'

        print(f'\nCreating {name}')
        net = create_graph(name, GA, start, end)
        filled = create_filled_graph(filled_name, net)

## Create Graphs

In [ ]:
global_time_periods = [
    (1946, 1955), # Post war, Korean War
    (1956, 1975), # Vietnam War
    (1976, 1990), # Late Cold War
    (1991, 2002), # Collapse of Communism
    (2003, 2024), # Contemporary
]

serbia_time_periods = [
    (1946, 1979), # Early Yugoslavia
    (1980, 1991), # Late Yugoslavia
    (1992, 2005), # Breakup of Yugoslavia
    (2006, 2024), # Modern Serbia
    (2012, 2024), # Punished Serbia
]

In [ ]:
# Load the chosen graph
graph_name = "net_ga_2003_2024_filled"
graph_path = "../graphs/{}.gml".format(graph_name)
net = nx.read_gml(graph_path)

In [ ]:
#net_ga_all = create_graph('net_ga_all', GA, START, END)
#net_sc_all = create_graph('net_sc_all', SC, START, END)
#net_ga_2006_end = create_graph('net_ga_2006_end', GA, 2006, END)

#net_ga_all_filled = create_filled_graph('net_ga_all_filled', net_ga_all)

#create_graphs_from_periods(global_time_periods)
#create_graphs_from_periods(serbia_time_periods, "_srb")